# Problema Mix de Produção

<div style="text-align:justify">
A definição do mix de produção proporciona a alocação dos recursos produtivos no processo de manufatura, visando a otimização da utilização dos recursos e do desempenho do sistema produtivo o que, por sua vez, a um nível gerencial, norteia o desempenho da organização.
</div>

## Variáveis

Seja:<br>
- p<sub>i</sub> = O luco do </i>i-ésimo</i> produto
- m<sub>j</sub> = A </i>j-ésima</i> máquina
- c<sub>ij</sub> = O custo de produzir p<sub>i</sub> na máquina m<sub>j</sub>
- a<sub>j</sub> = O tempo disponível para a máquina m<sub>j</sub>
- x<sub>i</sub> = x<sub>i</sub> &isin; &real; &frasl; x<sub>i</sub> &ge; 0

## Função Objetivo
<br>
Maximizar o lucro de produção.

$$ \sum_{i=0}^{n} x_i.p_i $$

## Restrição
<br>
$$ \sum_{i=0}^{n} x_i.c_{ij} \leq a_j, \forall j=0 ... m $$

## Implementação

<div style="text-align:justify">
Vamos implementar a solução modelada utilizando a Gurobi API. Para isso, vamos criar dados sintéticos que representem o lucro de produtos e seus respectivos custos de produção por máquina.
</div>

#### Carregando Dependências

In [1]:
import gurobipy

#### Produtos

In [2]:
prods = ['prod_{}'.format(i) for i in range(3)]
prods

['prod_0', 'prod_1', 'prod_2']

#### Lucos

In [3]:
profit = [12, 18, 22]
p = {}
for i, value in enumerate(prods):
    p[value] = profit[i]

p

{'prod_0': 12, 'prod_1': 18, 'prod_2': 22}

#### Máquinas

In [4]:
mach = ['mach_{}'.format(i) for i in range(3)]
mach

['mach_0', 'mach_1', 'mach_2']

#### Tempo Disponível por Máquina

In [5]:
time = [120, 200, 250]
a = {}
for i, value in enumerate(mach):
    a[value] = time[i]

a

{'mach_0': 120, 'mach_1': 200, 'mach_2': 250}

#### Custo de Produto por Máquina

In [6]:
cost = [
    [1.5, 0.0, 1.2],
    [0.0, 2.2, 2.0],
    [1.2, 1.4, 2.4]
]

c = {}
for i, prd_value in enumerate(prods):
    for j, mch_value in enumerate(mach):
        c[prd_value, mch_value] = cost[i][j]

print(c)

{('prod_0', 'mach_0'): 1.5, ('prod_0', 'mach_1'): 0.0, ('prod_0', 'mach_2'): 1.2, ('prod_1', 'mach_0'): 0.0, ('prod_1', 'mach_1'): 2.2, ('prod_1', 'mach_2'): 2.0, ('prod_2', 'mach_0'): 1.2, ('prod_2', 'mach_1'): 1.4, ('prod_2', 'mach_2'): 2.4}


#### Modelagem

In [7]:
model = gurobipy.Model()

x = model.addVars(prods)

model.setObjective(
    gurobipy.quicksum(x[i] * p[i] for i in prods),
    sense=gurobipy.GRB.MAXIMIZE
)

c1 = model.addConstrs(
    gurobipy.quicksum(x[i] * c[i,j] for i in prods) <= a[j] for j in mach
)

model.optimize()

Restricted license - for non-production use only - expires 2022-01-13
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 3 columns and 7 nonzeros
Model fingerprint: 0xa81d5c28
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Presolve time: 0.01s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.2000000e+31   5.000000e+30   5.200000e+01      0s
       5    2.3460000e+03   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds
Optimal objective  2.346000000e+03


#### Lucro Máximo

In [8]:
model.objVal

2346.0

#### Unidades de Produtos

In [9]:
for p in prods:
    print('Produto {}: {}'.format(p, x[p].X))

Produto prod_0: 80.0
Produto prod_1: 77.0
Produto prod_2: 0.0


#### Ocupação de Máquinas

In [10]:
for m in mach:
    print('Máquina {}: {} horas'.format(m, a[m] - c1[m].Slack))

Máquina mach_0: 120.0 horas
Máquina mach_1: 169.4 horas
Máquina mach_2: 250.0 horas
